### 1)-Importing Packages

In [1]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
//import org.apache.spark.ml.clustering.{KMeans, KMeansModel, GaussianMixture, GaussianMixtureModel, BisectingKMeans, BisectingKMeansModel}
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation.ClusteringEvaluator
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator,BinaryClassificationEvaluator}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
84,application_1576053140655_0085,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation.ClusteringEvaluator
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar


### 2)-Load Dataset

In [2]:
val raw = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").csv("datasets/creditcard.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

raw: org.apache.spark.sql.DataFrame = [Time: string, V1: string ... 29 more fields]


In [3]:
// cast all the column to Double type.
val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 29 more fields]


### 3)-Data Preparation for Training

In [4]:
val labelConverter = new StringIndexer().setInputCol("Class").setOutputCol("label")
val assembler = new VectorAssembler().setInputCols(Array("V3", "V4", "V9", "V10", "V11", "V12", "V14", "V16", "V17", "V18","V19")).setOutputCol("assembled")
val scaler = new MinMaxScaler().setInputCol("assembled").setOutputCol("features")
//val scaler = new StandardScaler().setInputCol("assembled").setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(assembler, scaler, labelConverter))
val pipelineModel = pipeline.fit(df)
val data = pipelineModel.transform(df)
println("Generate feature from raw data:")
data.select("features", "label").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.StringIndexer = strIdx_821b047ae2d9
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_132ba44725a8
scaler: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_fe93def8bc1a
pipeline: org.apache.spark.ml.Pipeline = pipeline_8083bf0968b3
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_8083bf0968b3
data: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 32 more fields]
Generate feature from raw data:
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.88136490328633...|  0.0|
|[0.84029849039390...|  0.0|
|[0.86814081926190...|  0.0|
|[0.86848364774806...|  0.0|
|[0.86425070140685...|  0.0|
|[0.85718742663956...|  0.0|
|[0.83819983804773...|  0.0|
|[0.85603110871019...|  0.0|
|[0.83545216735689...|  0.0|
|[0.85551101187934...|  0.0|
|[0.85324951634142...|  0.0|
|[0.82226325870284...|  0.0|
|[0.84406658980857...|  0.0|
|[0.85177230541965...|  0.0|
|[0.86586281889129

In [5]:
val data0 = data.filter($"Class" === "0").cache()
val data1 = data.filter($"Class" === "1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
data1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [6]:
val splitTime0 = data0.stat.approxQuantile("Time", Array(0.7), 0.001).head
//val splitTime1 = data1.stat.approxQuantile("Time", Array(0.2), 0.001).head
val trainingData0 = data0.filter(s"Time<$splitTime0").cache()
val validationData0 = data0.filter(s"Time>=$splitTime0").cache()
//val trainingData1 = data1.filter(s"Time<$splitTime1").cache()
//val validationData1 = data1.filter(s"Time>=$splitTime1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitTime0: Double = 132941.0
trainingData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [7]:
val trainingData1 = trainingData0.unionAll(data1)
val trainingData2 = 

val validationData = validationData0.unionAll(data1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [8]:
println(" Training set statistics: 1 represents fraud and 0 represents normal")
trainingData.groupBy("Class").count().show()
println(" validation set statistics: 1 represents fraud and 0 represents normal")
validationData.groupBy("Class").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Training set statistics: 1 represents fraud and 0 represents normal
+-----+------+
|Class| count|
+-----+------+
|  0.0|199006|
|  1.0|   492|
+-----+------+

 validation set statistics: 1 represents fraud and 0 represents normal
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85309|
|  1.0|  492|
+-----+-----+



### 4)-Training the models

In [9]:
// Create a Kmeans Model with K=2
// train the model
val kmeans = new KMeans().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)
val bisectingkmeans = new BisectingKMeans().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)
val gaussianMixture = new GaussianMixture().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_936fd95afb00
bisectingkmeans: org.apache.spark.ml.clustering.BisectingKMeans = bisecting-kmeans_c60eafa76633
gaussianMixture: org.apache.spark.ml.clustering.GaussianMixture = GaussianMixture_18b470ecfc65


In [10]:
// Fit that model to the training_data
val t = System.nanoTime
val modelKmeans = kmeans.fit(trainingData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 369500297928387
modelKmeans: org.apache.spark.ml.clustering.KMeansModel = kmeans_936fd95afb00
durationkmeans: Double = 13.282288086

initial model training finished.
Training process takes 13.282288086 secs


In [11]:
// Fit that model to the training_data
val t = System.nanoTime
val modelBisectingkmeans = bisectingkmeans.fit(trainingData)
val durationbisectingkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationbisectingkmeans secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 369515612519483
modelBisectingkmeans: org.apache.spark.ml.clustering.BisectingKMeansModel = bisecting-kmeans_c60eafa76633
durationbisectingkmeans: Double = 35.682767438

initial model training finished.
Training process takes 35.682767438 secs


In [12]:
// Fit that model to the training_data
val t = System.nanoTime
val modelGMM = gaussianMixture.fit(trainingData)
val durationGMM = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationGMM secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 369553213250874
modelGMM: org.apache.spark.ml.clustering.GaussianMixtureModel = GaussianMixture_18b470ecfc65
durationGMM: Double = 9.389699516

initial model training finished.
Training process takes 9.389699516 secs


### 5)- Prediction

In [13]:
 // Make predictions Kmeans
val t = System.nanoTime
var predictionsk = modelKmeans.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsk = predictionsk.select(col("label"),col("clusters").cast("Double"),col("features"))
//val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)
predictionsk.groupBy("clusters").count().show()
//predictionsk.groupBy("label").count().show()
predictionsk.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 369564518935412
predictionsk: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 33 more fields]
durationkmeans: Double = 0.225973311

initial model training finished.
Training process takes 0.225973311 secs
predictionsk: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|37121|
|     1.0|48680|
+--------+-----+

+-----+--------+--------------------+
|label|clusters|            features|
+-----+--------+--------------------+
|  0.0|     0.0|[0.82682350930621...|
|  0.0|     0.0|[0.86425421948073...|
|  0.0|     1.0|[0.81514790400608...|
|  0.0|     1.0|[0.84421006436973...|
|  0.0|     1.0|[0.80905773386549...|
|  0.0|     0.0|[0.83799136022426...|
|  0.0|     1.0|[0.85023895690320...|
|  0.0|     1.0|[0.83071382175165...|
|  0.0|     1.0|[0.80653200441440...|
|  0.0|     0.0|[0.85674920053228...|
+-----+--------+--------------------+
only showing top 10 rows



In [14]:
predictionsk.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|36694|
|     1.0|  0.0|48615|
|     0.0|  1.0|  427|
|     1.0|  1.0|   65|
+--------+-----+-----+



In [15]:
 // Make predictions Bisecting Kmeans 
val t = System.nanoTime
var predictionsbk = modelBisectingkmeans.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsbk = predictionsbk.select(col("label"),col("clusters").cast("Double"),col("features"))
predictionsbk.groupBy("clusters").count().show()
predictionsbk.groupBy("label").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 369570026895458
predictionsbk: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 33 more fields]
durationkmeans: Double = 0.229758156

initial model training finished.
Training process takes 0.229758156 secs
predictionsbk: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|48644|
|     1.0|37157|
+--------+-----+

+-----+-----+
|label|count|
+-----+-----+
|  0.0|85309|
|  1.0|  492|
+-----+-----+



In [16]:
predictionsbk.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|48578|
|     1.0|  0.0|36731|
|     0.0|  1.0|   66|
|     1.0|  1.0|  426|
+--------+-----+-----+



In [17]:
// Make predictions Bisecting Kmeans 
val t = System.nanoTime
var predictionsgmm = modelGMM.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsgmm = predictionsgmm.select(col("label"),col("clusters").cast("Double"),col("features"))
predictionsgmm.groupBy("clusters").count().show()
predictionsgmm.groupBy("label").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 369574523176013
predictionsgmm: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationkmeans: Double = 0.226458469

initial model training finished.
Training process takes 0.226458469 secs
predictionsgmm: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|41930|
|     1.0|43871|
+--------+-----+

+-----+-----+
|label|count|
+-----+-----+
|  0.0|85309|
|  1.0|  492|
+-----+-----+



In [18]:
predictionsgmm.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|41460|
|     1.0|  0.0|43849|
|     0.0|  1.0|  470|
|     1.0|  1.0|   22|
+--------+-----+-----+



### 6)-Evaluation

 ##### Calculation of Silhouette Score

In [19]:
// Evaluate clustering by computing Silhouette score
val evaluatorsi = new ClusteringEvaluator().setPredictionCol("clusters").setMetricName("silhouette")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorsi: org.apache.spark.ml.evaluation.ClusteringEvaluator = cluEval_82508b085180


In [20]:
val ScoreKmeans = evaluatorsi.evaluate(predictionsk)
println(s"Silhouette of Kmeans predictions with squared euclidean distance = $ScoreKmeans")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ScoreKmeans: Double = 0.2342085407532651
Silhouette of Kmeans predictions with squared euclidean distance = 0.2342085407532651


In [21]:
val ScoreBisectingKmeans  = evaluatorsi.evaluate(predictionsbk)
println(s"Silhouette of Bisecting Kmeans predictions with squared euclidean distance = $ScoreBisectingKmeans")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ScoreBisectingKmeans: Double = 0.23429057822234223
Silhouette of Bisecting Kmeans predictions with squared euclidean distance = 0.23429057822234223


In [22]:
val ScoreGMM  = evaluatorsi.evaluate(predictionsgmm)
println(s"Silhouette of Gaussian Mixture predictions with squared euclidean distance = $ScoreGMM")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ScoreGMM: Double = 0.10564208774701965
Silhouette of Gaussian Mixture predictions with squared euclidean distance = 0.10564208774701965


In [23]:
// Evaluate clustering.
val costKmeans = modelKmeans.computeCost(trainingData)
println(s"Kmeans: Within Set Sum of Squared Errors = $costKmeans")
val costBisectingKmeans = modelBisectingkmeans.computeCost(trainingData)
println(s"Bisecting Kmeans: Within Set Sum of Squared Errors = $costBisectingKmeans")
val costBisectingKmeans = modelBisectingkmeans.computeCost(trainingData)
println(s"Bisecting Kmeans: Within Set Sum of Squared Errors = $costBisectingKmeans")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

costKmeans: Double = 3879.4910293905696
Kmeans: Within Set Sum of Squared Errors = 3879.4910293905696
costBisectingKmeans: Double = 3879.4886782515073
Bisecting Kmeans: Within Set Sum of Squared Errors = 3879.4886782515073
costBisectingKmeans: Double = 3879.488678251507
Bisecting Kmeans: Within Set Sum of Squared Errors = 3879.488678251507


In [24]:
val evaluatork1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatork2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatork3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatork4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCk = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracyk = evaluatork1.evaluate(predictionsk)
println("Area Under ROC Curve = " + areaUnderROCk.evaluate(predictionsk))
println("Accuracy = " + evaluatork1.evaluate(predictionsk))
println("Precision = " + evaluatork2.evaluate(predictionsk))
println("Recall = " + evaluatork3.evaluate(predictionsk))
println("F1 = " + evaluatork4.evaluate(predictionsk))
println("Test Error = " + (1.0 - accuracyk))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatork1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_8dfc566dd3fd
evaluatork2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5aaae6584954
evaluatork3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_d88dc9a83682
evaluatork4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_439c23e300ae
areaUnderROCk: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_4f43357e4207
accuracyk: Double = 0.4284215801680633
Area Under ROC Curve = 0.28112214401458036
Accuracy = 0.4284215801680633
Precision = 0.9828364948643763
Recall = 0.4284215801680633
F1 = 0.5960061638947316
Test Error = 0.5715784198319367


In [25]:
val evaluatorbk1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatorbk2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatorbk3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatorbk4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCbk = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracybk = evaluatorbk1.evaluate(predictionsbk)
println("Area Under ROC Curve = " + areaUnderROCbk.evaluate(predictionsbk))
println("Accuracy = " + evaluatorbk1.evaluate(predictionsbk))
println("Precision = " + evaluatorbk2.evaluate(predictionsbk))
println("Recall = " + evaluatorbk3.evaluate(predictionsbk))
println("F1 = " + evaluatorbk4.evaluate(predictionsbk))
println("Test Error = " + (1.0 - accuracybk))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorbk1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_a53a9f9073c4
evaluatorbk2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_1032e95a1911
evaluatorbk3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_a853491025f8
evaluatorbk4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5d733e950425
areaUnderROCbk: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_c45c19251b2d
accuracybk: Double = 0.5711355345508794
Area Under ROC Curve = 0.7176447371092004
Accuracy = 0.5711355345508794
Precision = 0.9929825267397665
Recall = 0.5711355345508794
F1 = 0.7212699277792068
Test Error = 0.4288644654491206


In [26]:
val evaluatorgmm1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatorgmm2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatorgmm3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatorgmm4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCgmm = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracygmm = evaluatorgmm1.evaluate(predictionsgmm)
println("Area Under ROC Curve = " + areaUnderROCgmm.evaluate(predictionsgmm))
println("Accuracy = " + evaluatorgmm1.evaluate(predictionsgmm))
println("Precision = " + evaluatorgmm2.evaluate(predictionsgmm))
println("Recall = " + evaluatorgmm3.evaluate(predictionsgmm))
println("F1 = " + evaluatorgmm4.evaluate(predictionsgmm))
println("Test Error = " + (1.0 - accuracygmm))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorgmm1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_9f391668f623
evaluatorgmm2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_352e81f2cd5f
evaluatorgmm3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_ef873fda61c0
evaluatorgmm4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_21363f0c0289
areaUnderROCgmm: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_85d71f563fbc
accuracygmm: Double = 0.48346755865316254
Area Under ROC Curve = 0.2653567037551771
Accuracy = 0.48346755865316254
Precision = 0.9831237940493833
Recall = 0.48346755865316254
F1 = 0.6479557672624807
Test Error = 0.5165324413468375
